In [0]:
import tensorflow as tf
from tensorflow.python import control_flow_ops
import time, os

import numpy as np
import matplotlib.pyplot as plt
import glob
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from tflearn.data_utils import to_categorical, shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn import DNN
import tflearn
import cv2
tf.reset_default_graph()

%matplotlib inline

IMG_SIZE = 50

def plot_img(im):
    im_gray = rgb2gray(im)
    plt.figure(figsize=(10,4))

    plt.subplot(1,2,1)
    plt.imshow(im)
    plt.axis('off')
    plt.title('Original Image')

    plt.subplot(1,2,2)
    plt.imshow(im_gray, cmap='gray')
    plt.axis('off')
    plt.title('Grayscale Image')

    plt.tight_layout()
    plt.show()

def load_data(path):
    images, labels = [], []
    folders = os.listdir(path)
    for i in range(len(folders)):
        dir_img = os.path.join(path, folders[i])
        labels.append(i)
        for j in os.listdir(dir_img):
            img = os.path.join(dir_img, j)
            img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
            try:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                images.append(np.array(img))
            except:
                continue
    return np.array(images, dtype=np.float64), labels
if "learning-neural-networks" not in os.listdir():
  ! git clone https://github.com/agusmdev/learning-neural-networks

In [0]:
if "test-data" not in os.listdir():
  !git clone https://github.com/agusmdev/test-data

images, labels = load_data("./learning-neural-networks/datasets/flowers/")

images.shape

(4323, 50, 50)

In [0]:
""" Process our TF dataset """
# split our tf set in a test and training part

X, X_test, Y, Y_test = train_test_split(images, labels, test_size=0.1, random_state=42)
X, Y = shuffle(X, Y)

# encode our labels
Y = to_categorical(Y, 5)
Y_test = to_categorical(Y_test, 5)
X = X.reshape([-1, IMG_SIZE, IMG_SIZE, 1])
X_test = X_test.reshape([-1, IMG_SIZE, IMG_SIZE, 1])

X.shape

(3890, 50, 50, 1)

In [0]:
tf_img_prep = ImagePreprocessing()
tf_img_prep.add_featurewise_zero_center()
tf_img_prep.add_featurewise_stdnorm()


# Randomly create extra image data by rotating and flipping images
tf_img_aug = ImageAugmentation()
tf_img_aug.add_random_flip_leftright()
tf_img_aug.add_random_rotation(max_angle=30.)


In [0]:
#tf.reset_default_graph()
# Convolutional network building
network = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1],
                     data_preprocessing=tf_img_prep,
                     data_augmentation=tf_img_aug)
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 128, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 1024, activation='relu')
network = dropout(network, 0.8)
network = fully_connected(network, 5, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path='model.tfl.ckpt')
model.fit(X, Y, n_epoch=300, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96, run_id='flowers_cnn2', snapshot_epoch=True)
model.save("flowers_cnn2.model")
#model.load("flowers_cnn.model")

Training Step: 12299  | total loss: 0.02318 | time: 2.095s
| Adam | epoch: 300 | loss: 0.02318 - acc: 0.9931 -- iter: 3840/3890
Training Step: 12300  | total loss: 0.02274 | time: 3.151s
| Adam | epoch: 300 | loss: 0.02274 - acc: 0.9938 | val_loss: 2.01712 - val_acc: 0.5820 -- iter: 3890/3890
--
INFO:tensorflow:/content/model.tfl.ckpt-12300 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/content/flowers_cnn2.model is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
from tqdm import tqdm
#TEST_DIR = "./learning-neural-networks/datasets/flowers/tulip"
TEST_DIR = "./test-data/flowers_data/"
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img, dtype=np.float32), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data
  
labels = ["sunflower", "tulip", "rose", "dandelion", "daisy"] 
test_data = process_test_data()
fig=plt.figure()

for num,data in enumerate(test_data):
    
    img_num = data[1]
    img_data = data[0]
    
    y = fig.add_subplot(3,4,num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    #model_out = model.predict([data])[0]
    model_out = model.predict([data])[0]
    
    str_label = labels[np.argmax(model_out)]

        
    y.imshow(orig,cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()

In [0]:
image_path = './test-data/flowers_data/rose.jpg'
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
img = np.array(cv2.resize(img, (IMG_SIZE, IMG_SIZE)), dtype=np.float64)
#img = img.reshape([IMG_SIZE, IMG_SIZE, 1])
img = img.reshape(IMG_SIZE, IMG_SIZE, -1)
#plot_img(img)
img.shape
#X_test[0].shape
# img.shape
# # Predict
predicted = model.predict([img])
predicted

predicciones = np.array(model.predict(X_test)).argmax(axis=1)
correctas = Y_test.argmax(axis=1)
certeza = np.mean(predicciones == correctas, axis=0)
print("La certeza es de: ", certeza)

print("\n\n\nPrediccion:")
# Check the result.
print(labels[np.argmax(predicted[0])])
print("\nProbabilidades: "+str(np.around(predicted[0]*100, decimals=3)))
print("\n\n\n")
